In [1]:
import tensorflow as tf

In [2]:
training_samples_file_path = tf.keras.utils.get_file("trainingSamples.csv", "file:///D:/program/SparrowRecSys/src/main/resources/webroot/sampledata/traingSamples.csv")
test_samples_file_path = tf.keras.utils.get_file("testSamples.csv", "file:///D:/program/SparrowRecSys/src/main/resources/webroot/sampledata/testSamples.csv")

In [3]:
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name="label",
        na_value = "0",
        num_epochs=1,
        ignore_errors=True
    )
    return dataset

In [4]:
train_dataset = get_dataset(training_samples_file_path)
test_dataset = get_dataset(test_samples_file_path)

In [5]:
inputs = {
    'movieAvgRating': tf.keras.layers.Input(name='movieAvgRating', shape=(), dtype='float32'),
    'movieRatingStddev': tf.keras.layers.Input(name='movieRatingStddev', shape=(), dtype='float32'),
    'movieRatingCount': tf.keras.layers.Input(name='movieRatingCount', shape=(), dtype='int32'),
    'userAvgRating': tf.keras.layers.Input(name='userAvgRating', shape=(), dtype='float32'),
    'userRatingStddev': tf.keras.layers.Input(name='userRatingStddev', shape=(), dtype='float32'),
    'userRatingCount': tf.keras.layers.Input(name='userRatingCount', shape=(), dtype='int32'),
    'releaseYear': tf.keras.layers.Input(name='releaseYear', shape=(), dtype='int32'),

    'movieId': tf.keras.layers.Input(name='movieId', shape=(), dtype='int32'),
    'userId': tf.keras.layers.Input(name='userId', shape=(), dtype='int32'),
    'userRatedMovie1': tf.keras.layers.Input(name='userRatedMovie1', shape=(), dtype='int32'),

    'userGenre1': tf.keras.layers.Input(name='userGenre1', shape=(), dtype='string'),
    'userGenre2': tf.keras.layers.Input(name='userGenre2', shape=(), dtype='string'),
    'userGenre3': tf.keras.layers.Input(name='userGenre3', shape=(), dtype='string'),
    'userGenre4': tf.keras.layers.Input(name='userGenre4', shape=(), dtype='string'),
    'userGenre5': tf.keras.layers.Input(name='userGenre5', shape=(), dtype='string'),
    'movieGenre1': tf.keras.layers.Input(name='movieGenre1', shape=(), dtype='string'),
    'movieGenre2': tf.keras.layers.Input(name='movieGenre2', shape=(), dtype='string'),
    'movieGenre3': tf.keras.layers.Input(name='movieGenre3', shape=(), dtype='string'), 
}

In [6]:
# movie id embedding feature
movie_col = tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001)
movie_emb_col = tf.feature_column.embedding_column(movie_col, 10)
movie_ind_col = tf.feature_column.indicator_column(movie_col)

In [7]:
# user id embedding feature
user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, 10)
user_ind_col = tf.feature_column.indicator_column(user_col)

In [8]:
# genre features vocabulary
genre_vocab = [
    'Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary',
    'Sci-Fi', 'Drama', 'Thriller', 'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical'
]

In [9]:
# user genre embedding feature
user_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key='userGenre1', vocabulary_list=genre_vocab)
user_genre_ind_col = tf.feature_column.indicator_column(user_genre_col)
user_genre_emb_col = tf.feature_column.embedding_column(user_genre_col, 10)

In [10]:
# item genre embeddiing feature
item_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key='movieGenre1', vocabulary_list=genre_vocab)
item_genre_ind_col = tf.feature_column.indicator_column(item_genre_col)
item_genre_emb_col = tf.feature_column.embedding_column(item_genre_col, 10)

In [11]:
# fm first-order categorical items
cat_columns = [movie_ind_col, user_ind_col, user_genre_ind_col, item_genre_ind_col]

In [12]:
deep_columns = [
    tf.feature_column.numeric_column("releaseYear"),
    tf.feature_column.numeric_column("movieRatingCount"),
    tf.feature_column.numeric_column("movieAvgRating"),
    tf.feature_column.numeric_column("movieRatingStddev"),
    tf.feature_column.numeric_column("userRatingCount"),
    tf.feature_column.numeric_column("userAvgRating"),
    tf.feature_column.numeric_column("userRatingStddev")
]

In [13]:
first_order_cat_feature = tf.keras.layers.DenseFeatures(cat_columns)(inputs)
first_order_cat_feature = tf.keras.layers.Dense(1, activation=None)(first_order_cat_feature)
first_order_deep_feature = tf.keras.layers.DenseFeatures(deep_columns)(inputs)
first_order_deep_feature = tf.keras.layers.Dense(1, activation=None)(first_order_deep_feature)

In [14]:
first_order_feature = tf.keras.layers.Add()([first_order_cat_feature, first_order_deep_feature])

In [15]:
second_order_cat_columns_emb = [
    tf.keras.layers.DenseFeatures([item_genre_emb_col])(inputs),
    tf.keras.layers.DenseFeatures([movie_emb_col])(inputs),
    tf.keras.layers.DenseFeatures([user_genre_emb_col])(inputs),
    tf.keras.layers.DenseFeatures([user_emb_col])(inputs)
]

In [16]:
second_order_cat_columns = []
for feature_emb in second_order_cat_columns_emb:
    feature = tf.keras.layers.Dense(64, activation=None)(feature_emb)
    feature = tf.keras.layers.Reshape((-1, 64))(feature)
    second_order_cat_columns.append(feature)

In [17]:
second_order_deep_columns = tf.keras.layers.DenseFeatures(deep_columns)(inputs)
second_order_deep_columns = tf.keras.layers.Dense(64, activation=None)(second_order_deep_columns)
second_order_deep_columns = tf.keras.layers.Reshape((-1, 64))(second_order_deep_columns)
second_order_fm_feature = tf.keras.layers.Concatenate(axis=1)(second_order_cat_columns + [second_order_deep_columns])

In [18]:
# second_order_deep_feature
deep_feature = tf.keras.layers.Flatten()(second_order_fm_feature)
deep_feature = tf.keras.layers.Dense(32, activation='relu')(deep_feature)
deep_feature = tf.keras.layers.Dense(16, activation='relu')(deep_feature)

In [19]:
class ReduceLayer(tf.keras.layers.Layer):
    def __init__(self, axis, op='sum', **kwargs):
        super().__init__()
        self.axis = axis
        self.op = op
        assert self.op in ['sum', 'mean']
    
    def build(self, input_shape):
        pass
    
    def call(self, input, **kwargs):
        if self.op == 'sum':
            return tf.reduce_sum(input, axis=self.axis)
        elif self.op == 'mean':
            return tf.reduce_mean(input, axis=self.axis)
        return tf.reduce_sum(input, axis=self.axis)

In [20]:
second_order_sum_feature = ReduceLayer(1)(second_order_fm_feature)
second_order_sum_square_feature = tf.keras.layers.multiply([second_order_sum_feature, second_order_sum_feature])
second_order_square_feature = tf.keras.layers.multiply([second_order_fm_feature, second_order_fm_feature])
second_order_square_sum_feature = ReduceLayer(1)(second_order_square_feature)

In [21]:
second_order_fm_feature = tf.keras.layers.subtract([second_order_sum_square_feature, second_order_square_sum_feature])

In [22]:
concatenated_output = tf.keras.layers.Concatenate(axis=1)([first_order_feature, second_order_fm_feature, deep_feature])
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(concatenated_output)

In [23]:
model = tf.keras.Model(inputs, output_layer)

In [24]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy', tf.keras.metrics.AUC(curve='ROC'), tf.keras.metrics.AUC(curve='PR')]
)

In [26]:
# train the model
model.fit(train_dataset, epochs=5)

Epoch 1/5


C:\Users\EDZ\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\functional.py:543: UserWarning: Input dict contained keys ['rating', 'timestamp', 'userRatedMovie2', 'userRatedMovie3', 'userRatedMovie4', 'userRatedMovie5', 'userAvgReleaseYear', 'userReleaseYearStddev'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


7403/7403 [==============================] - 78s 11ms/step - loss: 7.5967 - accuracy: 0.5613 - auc: 0.5691 - auc_1: 0.6177
Epoch 2/5
7403/7403 [==============================] - 79s 11ms/step - loss: 0.7645 - accuracy: 0.6197 - auc: 0.6507 - auc_1: 0.6932
Epoch 3/5
7403/7403 [==============================] - 83s 11ms/step - loss: 0.6140 - accuracy: 0.6705 - auc: 0.7227 - auc_1: 0.7521
Epoch 4/5
7403/7403 [==============================] - 80s 11ms/step - loss: 0.5751 - accuracy: 0.7047 - auc: 0.7643 - auc_1: 0.7896
Epoch 5/5
7403/7403 [==============================] - 80s 11ms/step - loss: 0.5294 - accuracy: 0.7361 - auc: 0.8075 - auc_1: 0.8336


In [27]:
test_loss, test_accuracy, test_roc_auc, test_pr_auc = model.evaluate(test_dataset)
print('\n\n Test Loss {}, Test Accuracy {}, Test ROC AUC {}, Test PR AUC {}'.format(test_loss, test_accuracy, test_roc_auc, test_pr_auc))

C:\Users\EDZ\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\functional.py:543: UserWarning: Input dict contained keys ['rating', 'timestamp', 'userRatedMovie2', 'userRatedMovie3', 'userRatedMovie4', 'userRatedMovie5', 'userAvgReleaseYear', 'userReleaseYearStddev'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


1870/1870 [==============================] - 8s 4ms/step - loss: 0.6084 - accuracy: 0.6811 - auc: 0.7376 - auc_1: 0.7648


 Test Loss 0.6084297895431519, Test Accuracy 0.6811051964759827, Test ROC AUC 0.7376347780227661, Test PR AUC 0.7648492455482483
